# Analysis of RPM K5-Cre vs CGRP-Cre primary tumors
## Ireland et al 2024 BioRxiv
### Fig 1

### 1. Prep environment

In [ ]:
#Import other relevant packages
import numpy as np
import pandas as pd
from matplotlib import rcParams
import os
import scanpy as sc

import matplotlib as mpl
import matplotlib.pyplot as plt

#For nice color schemes
import cmocean

#For barplots
import seaborn as sns

In [ ]:
#Import scVI
import scvi
from scvi.model.utils import mde

scvi.settings.verbosity = 40

In [ ]:
sc.settings.verbosity = 3             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.settings.set_figure_params(dpi=80, facecolor='white')

In [ ]:
# Read 
os.chdir('/work/asi16')

### 2. Load in data and concatenate

In [ ]:
# Read in new RPMA TBO Allograft sample and re-aligned RPM TBO Allo samples
RPM1=sc.read_10x_mtx('RPM_CGRP/RPM_CGRP_15844X2_Custom_073024/outs/filtered_feature_bc_matrix', var_names='gene_symbols', cache=True)
RPM2=sc.read_10x_mtx('RPM_CGRP/RPM_CGRP_17520X1_CustomCount_073024/outs/filtered_feature_bc_matrix', var_names='gene_symbols', cache=True)
RPM3=sc.read_10x_mtx('RPM_CGRP/RPM_CGRP_17751X1_Custom2_073024/outs/filtered_feature_bc_matrix', var_names='gene_symbols', cache=True)
RPM4=sc.read_10x_mtx('RPM_CGRP/RPM_CGRP_17751X2_CustomCount2_073024/outs/filtered_feature_bc_matrix', var_names='gene_symbols', cache=True)
RPM5=sc.read_10x_mtx('082324_18571X2_RPMEarly/outs/filtered_feature_bc_matrix', var_names='gene_symbols', cache=True)

RPMK5t=sc.read_10x_mtx('07_2024_RPMK5/AI652_RPMK5_CustomCount_072324/outs/per_sample_outs/AI652_RPMK5_Tracheal/count/sample_filtered_feature_bc_matrix', var_names='gene_symbols', cache=True)
RPMK5c=sc.read_10x_mtx('07_2024_RPMK5/AI652_RPMK5_CustomCount_072324/outs/per_sample_outs/AI652_RPMK5_Central/count/sample_filtered_feature_bc_matrix', var_names='gene_symbols', cache=True)
RPMK5=sc.read_10x_mtx('07_2024_RPMK5/AI651_RPMK5_Total_CustomCount_072424/outs/filtered_feature_bc_matrix', var_names='gene_symbols', cache=True)


In [ ]:
# Add metadata to allografts
RPM1.obs['Genotype'] = 'RPM'
RPM1.obs['Cre'] = 'CGRP'
RPM1.obs['UnID'] = 'RPM_CGRP'
RPM1.obs['GenoCre'] = 'RPM_CGRP'
RPM1.obs['UnID'] = 'RPM_CGRP1'
RPM1.obs['Batch'] = 'RPM_CGRP1'

RPM2.obs['Genotype'] = 'RPM'
RPM2.obs['Cre'] = 'CGRP'
RPM2.obs['UnID'] = 'RPM_CGRP'
RPM2.obs['GenoCre'] = 'RPM_CGRP'
RPM2.obs['UnID'] = 'RPM_CGRP2'
RPM2.obs['Batch'] = 'RPM_CGRP2'

RPM3.obs['Genotype'] = 'RPM'
RPM3.obs['Cre'] = 'CGRP'
RPM3.obs['UnID'] = 'RPM_CGRP'
RPM3.obs['GenoCre'] = 'RPM_CGRP'
RPM3.obs['UnID'] = 'RPM_CGRP3'
RPM3.obs['Batch'] = 'RPM_CGRP3'

RPM4.obs['Genotype'] = 'RPM'
RPM4.obs['Cre'] = 'CGRP'
RPM4.obs['UnID'] = 'RPM_CGRP'
RPM4.obs['GenoCre'] = 'RPM_CGRP'
RPM4.obs['UnID'] = 'RPM_CGRP4'
RPM4.obs['Batch'] = 'RPM_CGRP4'

RPM5.obs['Genotype'] = 'RPM'
RPM5.obs['Cre'] = 'CGRP'
RPM5.obs['GenoCre'] = 'RPM_CGRP'
RPM5.obs['UnID'] = 'RPM_CGRP5'
RPM5.obs['Batch'] = 'RPM_CGRP5'


RPMK5t.obs['Genotype'] = 'RPM'
RPMK5t.obs['Cre'] = 'K5'
RPMK5t.obs['GenoCre'] = 'RPM_K5'
RPMK5t.obs['UnID'] = 'RPM_K5t'
RPMK5t.obs['Batch'] = 'RPM_K5_multi'

RPMK5c.obs['Genotype'] = 'RPM'
RPMK5c.obs['Cre'] = 'K5'
RPMK5c.obs['GenoCre'] = 'RPM_K5'
RPMK5c.obs['UnID'] = 'RPM_K5c'
RPMK5c.obs['Batch'] = 'RPM_K5_multi'

RPMK5.obs['Genotype'] = 'RPM'
RPMK5.obs['Cre'] = 'K5'
RPMK5.obs['GenoCre'] = 'RPM_K5'
RPMK5.obs['UnID'] = 'RPM_K5_total'
RPMK5.obs['Batch'] = 'RPM_K5_total'


In [ ]:
#Concatenate datasets and check groupings 
adata= RPM1.concatenate([RPM2, RPM3, RPM4, RPM5, RPMK5t, RPMK5c, RPMK5], index_unique=None, join="outer")

adata.obs.groupby(["UnID"]).apply(len)

In [ ]:
adata.obs.groupby(["GenoCre"]).apply(len)


In [ ]:
adata.obs.groupby(["Batch"]).apply(len)

In [ ]:
adata

### 3. Perform QC 

In [ ]:
#QC filtering
adata.var['mito'] = adata.var_names.str.startswith('mt-')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(adata, qc_vars=['mito'], percent_top=None, log1p=False, inplace=True)

In [ ]:
#QC filtering RPMA RPM only scanpy flow
sc.pp.filter_cells(adata, min_genes=200)

adata.var['mito'] = adata.var_names.str.startswith('mt-')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(adata, qc_vars=['mito'], percent_top=None, log1p=False, inplace=True)

In [ ]:
sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts', 'pct_counts_mito'],
             jitter=0.4, multi_panel=True)
sc.pl.scatter(adata, x='total_counts', y='pct_counts_mito')
sc.pl.scatter(adata, x='total_counts', y='n_genes_by_counts')

In [ ]:
#Filter data by slicing anndata object
adata = adata[adata.obs.n_genes_by_counts < 10000, :]
adata = adata[adata.obs.n_genes_by_counts > 2000, :]
adata = adata[adata.obs.total_counts > 1000, :]
adata = adata[adata.obs.pct_counts_mito < 15, :]

In [ ]:
#Prep for HVG and scvi
#log1p the data
adata.obs["log1p_total_counts"] = np.log1p(adata.obs["total_counts"])

In [ ]:
#Create layers
adata.layers["counts"] = adata.X.copy()
adata.layers['norm'] = adata.X.copy(); sc.pp.normalize_total(adata, target_sum=1e4, layer="norm")

### 4. Set up and train model (scvi)

In [ ]:
# ID HVG via Scanpy

sc.pp.highly_variable_genes(
    adata,n_top_genes=10000,
    subset=False,
    layer="counts",
    flavor="seurat_v3",
    batch_key="Batch"
)

In [ ]:
# Prepare model with appropriate batch key (using all genes first)

scvi.model.SCVI.setup_anndata(
    adata,
    layer="counts",
    batch_key='Batch',
    continuous_covariate_keys=["pct_counts_mito"]
)


In [ ]:
# Train and run model
model = scvi.model.SCVI(adata)

In [ ]:
model.train()

In [ ]:
#Fit model to data
#Get latent representation of model to apply to UMAP
latent = model.get_latent_representation()

adata.obsm["X_scVI_1.1"] = latent

### 5. Perform leiden clustering

In [ ]:
#Calculate neighbors using scVI model input
sc.pp.neighbors(adata, use_rep="X_scVI_1.1")
sc.tl.umap(adata, min_dist=0.5)

#Run leiden clustering based on neighbors
sc.tl.leiden(adata, key_added="leiden_scVI_1.1", resolution=1)

In [ ]:
#QC UMAPs
sc.pl.umap(
    adata,
    color=["n_genes_by_counts", "total_counts", "pct_counts_mito", "log1p_total_counts"],
    cmap="cubehelix_r",
    s=3,
    ncols=2,
)

In [ ]:
# Other UMAPs

fig, ax = plt.subplots(figsize=(8, 6))
sc.pl.umap(adata, color="Genotype", cmap="cmo.matter", s=30, ax=ax, vmax="p99.99", frameon=False, save=False)
fig, ax = plt.subplots(figsize=(8, 6))
sc.pl.umap(adata, color="leiden_scVI_1.1", legend_loc="on data", legend_fontsize='large',ax=ax, s=30, frameon=False, save=False)
fig, ax = plt.subplots(figsize=(8, 6))
sc.pl.umap(adata, color="GenoCre", legend_loc="right margin", ax=ax, s=30, frameon=False, save=False, palette=['purple','orange'])
fig, ax = plt.subplots(figsize=(8, 6))
sc.pl.umap(adata, color="Batch", legend_loc="right margin", ax=ax, s=30, frameon=False, save=False)


#Additional QC bar graphs
adata.obs['cluster'] = adata.obs["leiden_scVI_1.1"].copy()

#Plot Log1p total counts
fig, ax = plt.subplots(figsize=(20,6))
sns.boxenplot(data=adata.obs, x="cluster", y="log1p_total_counts", ax=ax)

#Plot Pct counts mito
fig, ax = plt.subplots(figsize=(20,6))
sns.boxenplot(data=adata.obs, x="cluster", y="pct_counts_mito", ax=ax)

In [ ]:
adata.write_h5ad("082424_adata_RPM_K5vCGRP.h5ad")

In [ ]:
adata=sc.read_h5ad("082324_adata_RPM_K5vCGRP.h5ad")

### 6. Visualize gene expression, top DEGs, QC metrics and remove non-tumor clusters

In [ ]:
#Feature plots for key non-tumor and tumor genes

more_types=["Ptprc","Pecam1","Acta2", #immune
             "Venus","fLuc", "Top2a","Mki67", #Prolif/Tumor
            'Ascl1','Neurod1','Pou2f3'] # SCLC subtype/Tumor markers

sc.pl.umap(
    adata,
    color=more_types,
    use_raw=False,
    legend_loc= "on data",
    color_map="cmo.dense",
    ncols=4,
    frameon=False,
    layer="norm",
    save=False
)

In [ ]:
# Find cluster markers for each leiden cluster to aid filtering
sc.tl.rank_genes_groups(adata, 'leiden_scVI_1.1', method='wilcoxon', layer='norm', use_raw=False)
pd.DataFrame(adata.uns['rank_genes_groups']['names']).head(50)

In [ ]:
#Identify and subset out low qual, plot doublets, and obvious non-tumor clusters (ptprc+)
# Immune is 27, 17, 16, 28, 26, 5, 9
#Endothelial is 12, 25

bad_clust=['27','17','16','28','26','5','9','12','25']

#Filter out bad/non-tumor clusters
to_keep=(~adata.obs['leiden_scVI_1.1'].isin(bad_clust))

#Copy over to new anndata object
adata_2 = adata[to_keep].copy()

#### From here, continue iterating through runs of scvi modeling until no clear low quality cell clusters or non-tumor cells are observed.
#### Start back up at "set up and train scvi model" and run through subsetting out "bad clusters". 
#### Each time clusters are removed, model is run again to recluster.

## ITERATION 2

In [ ]:
# ID HVG via Scanpy

sc.pp.highly_variable_genes(
    adata_2,n_top_genes=10000,
    subset=False,
    layer="counts",
    flavor="seurat_v3",
    batch_key="Batch"
)

In [ ]:
# Set up model and train

scvi.model.SCVI.setup_anndata(
    adata_2,
    layer="counts",
    batch_key='Batch',
    continuous_covariate_keys=["pct_counts_mito"]
)
model = scvi.model.SCVI(adata_2)
model.train()

In [ ]:
#Fit model to data
#Get latent representation of model to apply to UMAP
latent = model.get_latent_representation()

adata_2.obsm["X_scVI_1.2"] = latent

In [ ]:
#Calculate neighbors using scVI model input
sc.pp.neighbors(adata_2, use_rep="X_scVI_1.2")
sc.tl.umap(adata_2, min_dist=0.5)

#Run leiden clustering based on neighbors
sc.tl.leiden(adata_2, key_added="leiden_scVI_1.2", resolution=1)

In [ ]:
#QC UMAPs
sc.pl.umap(
    adata_2,
    color=["n_genes_by_counts", "total_counts", "pct_counts_mito", "log1p_total_counts"],
    cmap="cubehelix_r",
    s=3,
    ncols=2,
)

In [ ]:
# Feature plots

more_types=["Ptprc","Pecam1","Acta2","fLuc", "Top2a","Mki67",
            'Ascl1','Neurod1','Pou2f3','EGFP'] # Tumor markers

sc.pl.umap(
    adata_2,
    color=more_types,
    use_raw=False,
    legend_loc= "on data",
    color_map="cmo.dense",
    ncols=4,
    frameon=False,
    layer="norm",
    save=False
)

In [ ]:
adata_2.write_h5ad("08.24.24_RPM_K5vCGRP_adata2.h5ad")

In [ ]:
adata_2=sc.read_h5ad("08.24.24_RPM_K5vCGRP_adata2.h5ad")

In [ ]:
# Plot additional UMAPs

fig, ax = plt.subplots(figsize=(6, 6))
sc.pl.umap(adata_2, color="Genotype", cmap="cmo.matter", s=30, ax=ax, vmax="p99.99", frameon=False, save=False)
fig, ax = plt.subplots(figsize=(6, 6))
sc.pl.umap(adata_2, color="leiden_scVI_1.2", legend_loc="on data", legend_fontsize='large',ax=ax, s=30, frameon=False, save=False)
fig, ax = plt.subplots(figsize=(6, 6))
sc.pl.umap(adata_2, color="GenoCre", legend_loc="right margin", ax=ax, s=30, frameon=False, save=False, palette=['purple','orange'])
fig, ax = plt.subplots(figsize=(6, 6))
sc.pl.umap(adata_2, color="UnID", legend_loc="right margin", ax=ax, s=30, frameon=False, save=False)



#Additional QC bar graphs
adata_2.obs['cluster'] = adata_2.obs["leiden_scVI_1.2"].copy()

#Plot Log1p total counts
fig, ax = plt.subplots(figsize=(20,6))
sns.boxenplot(data=adata_2.obs, x="cluster", y="log1p_total_counts", ax=ax)

#Plot Pct counts mito
fig, ax = plt.subplots(figsize=(20,6))
sns.boxenplot(data=adata_2.obs, x="cluster", y="pct_counts_mito", ax=ax)

In [ ]:
# Check more extensively for non-tumor cells
more_types=["Col14a1", "Acta2","Myh11","Tagln","Mustn1", #fibroblast
              "Lpl","Lipa","Pparg","Plin2","Ear1","Fabp1","Spp1", #lipofibroblast/osteoblastic
              "Ptprc","Mertk","Marco","Mrc1","Ly75","Adgre1","Itgax","Cd68","Csf1r","Mafb","Msr1","Arg1","Adgre4","Clec4a1", #Macs/Myeloid
              "Cx3cr1","Itgam","Cd14", #Monocytes
              "S100a9","S100a8","Mmp9","Csf3r","Cxcr2","Ly6g", #Neuts
              "Batf3","Xcr1","Clec9a","Ccl17","Ccl22", #DC
              "Cd3d","Cd3e","Cd3g","Cd28","Cd8a","Cd4","Foxp3", # Tcell
              "Gzma","Ncr1","Gzmb", #NK
              "Fcmr","Cd19","Fcer2a","Pax5","Cd22","Cd79b","Cd79a", #B cells
              "Slamf7", "Prdm1", #Plasma
              "Mcam","Pecam1","Icam2","Cd36","Cd93", #Endothelial
    "Pdpn","Cav1","Cav2","Hopx","Timp3","Sema3f","Serpine1", #AT1
              "Abca3","Muc1","Sftpa1","Sftpb","Sftpd","Scd1", #AT2
              "Scgb1a1","Cyp2f2","Scgb3a2", "Scgb3a1","Lypd2",#Club
              "Muc5ac","Muc5b", # Goblet
              "Tubb4a","Foxa3","Foxj1","Rfx2","Rfx3","Trp73", #Ciliated
              'Krt5', 'Krt17','Krt15','Trp63','Id1','Icam1','Epas1','Aqp3','Sfn','Perp','Fxyd3','Sdc1','Gstm2','F3','Abi3bp','Adh7', # Basal
              'Bex2','Ascl1','Meis2','Hes6','Hoxb5','Foxa2','Sox4','Rora','Isl1','Id4', 'Neurod1','Neurod4','Nhlh1','Nhlh2',#NE/neuronal
              'Pou2f3','Trpm5','Ascl2','Ehf',
              'Lrmp','Gng13','Ltc4s','Alox5ap','Avil','Alox5','Atp2a3','Plk2', #tuft
              "Cftr","Ascl3", 'Stap1','Atp6v1c2','Pparg','Rasd1','Slc12a2', #ionocyte
              "Gja1","Nkx2-1","Epcam", # Lung lineage
              'Yap1','Wwtr1','Sox2','Cd44','Hes1', # Stem-like
             "Venus","fLuc", "Top2a","Mki67"] # Tumor markers

sc.pl.umap(
    adata_2,
    color=more_types,
    use_raw=False,
    legend_loc= "on data",
    color_map="cmo.dense",
    ncols=4,
    vmax="p99.5",s=50,
    frameon=False,
    layer="norm",
    save=False
)

In [ ]:
# Find cluster markers for each leiden cluster to aid filtering
sc.tl.rank_genes_groups(adata_2, 'leiden_scVI_1.2', method='wilcoxon', layer='norm', use_raw=False)
pd.DataFrame(adata_2.uns['rank_genes_groups']['names']).head(50)

In [ ]:
# Fibroblasts/endothelial C20 C14, C21
# Immune is C6, 
# Regular lung cells s C17, C22

bad_clust=['20','14','21','6','17','22']

#Filter out bad clusters
to_keep=(~adata_2.obs['leiden_scVI_1.2'].isin(bad_clust))

#Copy over to new anndata object
adata_3 = adata_2[to_keep].copy()

In [ ]:
adata_3

## ITERATION 3

In [ ]:
# ID HVGs within new subsetted data via Scanpy

sc.pp.highly_variable_genes(
    adata_3,n_top_genes=10000,
    subset=False,
    layer="counts",
    flavor="seurat_v3",
    batch_key='Batch'
)


In [ ]:
# Set up model

scvi.model.SCVI.setup_anndata(
    adata_3,
    layer="counts",
    batch_key='Batch',
    continuous_covariate_keys=["pct_counts_mito"]
)
model = scvi.model.SCVI(adata_3)
model.train()

In [ ]:
#Fit model to data
#Get latent representation of model to apply to UMAP
latent = model.get_latent_representation()

adata_3.obsm["X_scVI_1.3"] = latent

In [ ]:
#Calculate neighbors using scVI model input
sc.pp.neighbors(adata_3, use_rep="X_scVI_1.3")
sc.tl.umap(adata_3, min_dist=0.5)

#Run leiden clustering based on neighbors
sc.tl.leiden(adata_3, key_added="leiden_scVI_1.3", resolution=1)

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))
sc.pl.umap(adata_3, color="leiden_scVI_1.3", legend_loc="on data", legend_fontsize='large',ax=ax, s=50, frameon=False, save=False)


In [ ]:
adata_3.obs.groupby(["GenoCre"]).apply(len)

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))
sc.pl.umap(adata_3, color="Batch", cmap="cmo.matter", s=10, ax=ax, vmax="p99.99", frameon=False, save=False)
fig, ax = plt.subplots(figsize=(6, 6))
sc.pl.umap(adata_3, color="leiden_scVI_1.3", legend_loc="on data", legend_fontsize='large',ax=ax, s=10, frameon=False, save=False)
fig, ax = plt.subplots(figsize=(6, 6))
sc.pl.umap(adata_3, color="GenoCre", legend_loc="right margin", ax=ax, s=10, frameon=False, save=False, palette=['purple','orange'])
fig, ax = plt.subplots(figsize=(6, 6))
sc.pl.umap(adata_3, color="UnID", legend_loc="right margin", ax=ax, s=10, frameon=False, save=False)



#Additional QC bar graphs
adata_3.obs['cluster'] = adata_3.obs["leiden_scVI_1.3"].copy()

#Plot Log1p total counts
fig, ax = plt.subplots(figsize=(20,6))
sns.boxenplot(data=adata_3.obs, x="cluster", y="log1p_total_counts", ax=ax)

#Plot Pct counts mito
fig, ax = plt.subplots(figsize=(20,6))
sns.boxenplot(data=adata_3.obs, x="cluster", y="pct_counts_mito", ax=ax)

In [ ]:
more_types=["Col14a1", "Acta2","Myh11","Tagln","Mustn1", #fibroblast
              "Lpl","Lipa","Pparg","Plin2","Ear1","Fabp1","Spp1", #lipofibroblast/osteoblastic
              "Ptprc","Mertk","Marco","Mrc1","Ly75","Adgre1","Itgax","Cd68","Csf1r","Mafb","Msr1","Arg1","Adgre4","Clec4a1", #Macs/Myeloid
              "Cx3cr1","Itgam","Cd14", #Monocytes
              "S100a9","S100a8","Mmp9","Csf3r","Cxcr2","Ly6g", #Neuts
              "Batf3","Xcr1","Clec9a","Ccl17","Ccl22", #DC
              "Cd3d","Cd3e","Cd3g","Cd28","Cd8a","Cd4","Foxp3", # Tcell
              "Gzma","Ncr1","Gzmb", #NK
              "Fcmr","Cd19","Fcer2a","Pax5","Cd22","Cd79b","Cd79a", #B cells
              "Slamf7", "Prdm1", #Plasma
              "Mcam","Pecam1","Icam2","Cd36","Cd93", #Endothelial
    "Pdpn","Cav1","Cav2","Hopx","Timp3","Sema3f","Serpine1", #AT1
              "Abca3","Muc1","Sftpa1","Sftpb","Sftpd","Scd1", #AT2
              "Scgb1a1","Cyp2f2","Scgb3a2", "Scgb3a1","Lypd2",#Club
              "Muc5ac","Muc5b", # Goblet
              "Tubb4a","Foxa3","Foxj1","Rfx2","Rfx3","Trp73", #Ciliated
              'Krt5', 'Krt17','Krt15','Trp63','Id1','Icam1','Epas1','Aqp3','Sfn','Perp','Fxyd3','Sdc1','Gstm2','F3','Abi3bp','Adh7', # Basal
              'Bex2','Ascl1','Meis2','Hes6','Hoxb5','Foxa2','Sox4','Rora','Isl1','Id4', 'Neurod1','Neurod4','Nhlh1','Nhlh2',#NE/neuronal
              'Pou2f3','Trpm5','Ascl2','Ehf',
              'Lrmp','Gng13','Ltc4s','Alox5ap','Avil','Alox5','Atp2a3','Plk2', #tuft
              "Cftr","Ascl3", 'Stap1','Atp6v1c2','Pparg','Rasd1','Slc12a2', #ionocyte
              "Gja1","Nkx2-1","Epcam", # Lung lineage
              'Yap1','Wwtr1','Sox2','Cd44','Hes1', # Stem-like
             "Venus","fLuc", "Top2a","Mki67"] # Tumor markers

sc.pl.umap(
    adata_3,
    color=more_types,
    use_raw=False,
    legend_loc= "on data",
    color_map="cmo.dense",
    ncols=4,
    vmax="p99.5",s=50,
    frameon=False,
    layer="norm",
    save=False
)

In [ ]:
adata_3

In [ ]:
# Find cluster markers for each leiden cluster to aid filtering
sc.tl.rank_genes_groups(adata_3, 'leiden_scVI_1.3', method='wilcoxon', layer='norm', use_raw=False)
pd.DataFrame(adata_3.uns['rank_genes_groups']['names']).head(50)

In [ ]:
adata_3.write_h5ad("082524_RPM_CGRPvK5.h5ad")

In [ ]:
adata_3=sc.read_h5ad("082424_RPM_CGRPvK5.h5ad")

In [ ]:
#Identify and subset out low qual, plot doublets, and obvious non-tumor clusters (ptprc+)
# Immune/fibro 10 20

bad_clust=['10','20']

#Filter out bad clusters
to_keep=(~adata_3.obs['leiden_scVI_1.3'].isin(bad_clust))

#Copy over to new anndata object
adata_4 = adata_3[to_keep].copy()

## ITERATION 4

In [ ]:
#HVG via Scanpy
#Note here that if you run with a batch_key with few cells, will get b'reciprocal condition number error
sc.pp.highly_variable_genes(
    adata_4,n_top_genes=10000,
    subset=False,
    layer="counts",
    flavor="seurat_v3",
    batch_key="Batch"
)

In [ ]:
scvi.model.SCVI.setup_anndata(
    adata_4,
    layer="counts",
    batch_key='Batch',
    continuous_covariate_keys=["pct_counts_mito"]
)


model = scvi.model.SCVI(adata_4)
model.train()

In [ ]:
#Fit model to data
#Get latent representation of model to apply to UMAP
latent = model.get_latent_representation()

adata_4.obsm["X_scVI_1.4"] = latent

In [ ]:
#Calculate neighbors using scVI model input
sc.pp.neighbors(adata_4, use_rep="X_scVI_1.4")
sc.tl.umap(adata_4, min_dist=0.5)

#Run leiden clustering based on neighbors
sc.tl.leiden(adata_4, key_added="leiden_scVI_1.4", resolution=1)

In [ ]:
#QC UMAPs
sc.pl.umap(
    adata_4,
    color=["n_genes_by_counts", "total_counts", "pct_counts_mito", "log1p_total_counts"],
    cmap="cubehelix_r",
    s=3,
    ncols=2,
)

In [ ]:
adata_4.write_h5ad("08.24.24_RPM_K5vCGRP_adata4.h5ad")

In [ ]:
more_types=["Col14a1", "Acta2","Myh11","Tagln","Mustn1", #fibroblast
              "Lpl","Lipa","Pparg","Plin2","Ear1","Fabp1","Spp1", #lipofibroblast/osteoblastic
              "Ptprc","Mertk","Marco","Mrc1","Ly75","Adgre1","Itgax","Cd68","Csf1r","Mafb","Msr1","Arg1","Adgre4","Clec4a1", #Macs/Myeloid
              "Cx3cr1","Itgam","Cd14", #Monocytes
              "S100a9","S100a8","Mmp9","Csf3r","Cxcr2","Ly6g", #Neuts
              "Batf3","Xcr1","Clec9a","Ccl17","Ccl22", #DC
              "Cd3d","Cd3e","Cd3g","Cd28","Cd8a","Cd4","Foxp3", # Tcell
              "Gzma","Ncr1","Gzmb", #NK
              "Fcmr","Cd19","Fcer2a","Pax5","Cd22","Cd79b","Cd79a", #B cells
              "Slamf7", "Prdm1", #Plasma
              "Mcam","Pecam1","Icam2","Cd36","Cd93", #Endothelial
    "Pdpn","Cav1","Cav2","Hopx","Timp3","Sema3f","Serpine1", #AT1
              "Abca3","Muc1","Sftpa1","Sftpb","Sftpd","Scd1", #AT2
              "Scgb1a1","Cyp2f2","Scgb3a2", "Scgb3a1","Lypd2",#Club
              "Muc5ac","Muc5b", # Goblet
              "Tubb4a","Foxa3","Foxj1","Rfx2","Rfx3","Trp73", #Ciliated
              'Krt5', 'Krt17','Krt15','Trp63','Id1','Icam1','Epas1','Aqp3','Sfn','Perp','Fxyd3','Sdc1','Gstm2','F3','Abi3bp','Adh7', # Basal
              'Bex2','Ascl1','Meis2','Hes6','Hoxb5','Foxa2','Sox4','Rora','Isl1','Id4', 'Neurod1','Neurod4','Nhlh1','Nhlh2',#NE/neuronal
              'Pou2f3','Trpm5','Ascl2','Ehf',
              'Lrmp','Gng13','Ltc4s','Alox5ap','Avil','Alox5','Atp2a3','Plk2', #tuft
              "Cftr","Ascl3", 'Stap1','Atp6v1c2','Pparg','Rasd1','Slc12a2', #ionocyte
              "Gja1","Nkx2-1","Epcam", # Lung lineage
              'Yap1','Wwtr1','Sox2','Cd44','Hes1', # Stem-like
             "Venus","fLuc", "Top2a","Mki67"] # Tumor markers

sc.pl.umap(
    adata_4,
    color=more_types,
    use_raw=False,
    legend_loc= "on data",
    color_map="cmo.dense",
    ncols=4,
    vmax="p99.5",s=50,
    frameon=False,
    layer="norm",
    save=False
)

In [ ]:
# Find cluster markers for each leiden cluster to aid filtering
sc.tl.rank_genes_groups(adata_4, 'leiden_scVI_1.4', method='wilcoxon', layer='norm', use_raw=False)
pd.DataFrame(adata_4.uns['rank_genes_groups']['names']).head(50)

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))
sc.pl.umap(adata_4, color="Batch", cmap="cmo.matter", s=10, ax=ax, vmax="p99.99", frameon=False, save=False)
fig, ax = plt.subplots(figsize=(6, 6))
sc.pl.umap(adata_4, color="leiden_scVI_1.4", legend_loc="on data", legend_fontsize='large',ax=ax, s=10, frameon=False, save=False)
fig, ax = plt.subplots(figsize=(6, 6))
sc.pl.umap(adata_4, color="GenoCre", legend_loc="right margin", ax=ax, s=10, frameon=False, save=False, palette=['purple','orange'])
fig, ax = plt.subplots(figsize=(6, 6))
sc.pl.umap(adata_4, color="UnID", legend_loc="right margin", ax=ax, s=10, frameon=False, save=False)



#Additional QC bar graphs
adata_4.obs['cluster'] = adata_4.obs["leiden_scVI_1.4"].copy()

#Plot Log1p total counts
fig, ax = plt.subplots(figsize=(20,6))
sns.boxenplot(data=adata_4.obs, x="cluster", y="log1p_total_counts", ax=ax)

#Plot Pct counts mito
fig, ax = plt.subplots(figsize=(20,6))
sns.boxenplot(data=adata_4.obs, x="cluster", y="pct_counts_mito", ax=ax)

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))
sc.pl.umap(adata_4, color="leiden_scVI_1.4", legend_loc="on data", legend_fontsize='large',ax=ax, s=10, frameon=False, save=False)


In [ ]:
#Identify and subset out low qual, plot doublets, and obvious non-tumor clusters (ptprc+)
# Immune is 14 (T-cell)

bad_clust=['14']

#Filter out bad clusters
to_keep=(~adata_4.obs['leiden_scVI_1.4'].isin(bad_clust))

#Copy over to new anndata object
adata_5 = adata_4[to_keep].copy()

## ITERATION 5

In [ ]:
# ID HVG via Scanpy
sc.pp.highly_variable_genes(
    adata_5,n_top_genes=10000,
    subset=False,
    layer="counts",
    flavor="seurat_v3",
    batch_key="Batch"
)

In [ ]:
scvi.model.SCVI.setup_anndata(
    adata_5,
    layer="counts",
    batch_key='Batch',
    continuous_covariate_keys=["pct_counts_mito"]
)
model = scvi.model.SCVI(adata_5)
model.train()

In [ ]:
#Fit model to data
#Get latent representation of model to apply to UMAP
latent = model.get_latent_representation()

adata_5.obsm["X_scVI_1.5"] = latent

In [ ]:
#Fit model to data
#Get latent representation of model to apply to UMAP
latent = model.get_latent_representation()

adata_5.obsm["X_scVI_1.5"] = latent
#Calculate neighbors using scVI model input
sc.pp.neighbors(adata_5, use_rep="X_scVI_1.5")
sc.tl.umap(adata_5, min_dist=0.75)

#Run leiden clustering based on neighbors
sc.tl.leiden(adata_5, key_added="leiden_scVI_1.5", resolution=0.5)

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))
sc.pl.umap(adata_5, color="leiden_scVI_1.5", legend_loc="on data", legend_fontsize='large',ax=ax, s=10, frameon=False, save=False)
fig, ax = plt.subplots(figsize=(6, 6))
sc.pl.umap(adata_5, color="UnID", legend_loc="right margin", ax=ax, s=10, frameon=False, save=False)
fig, ax = plt.subplots(figsize=(6, 6))
sc.pl.umap(adata_5, color="UnID", legend_loc="right margin", ax=ax, s=10, frameon=False, save=False)



#Additional QC bar graphs
adata_5.obs['cluster'] = adata_5.obs["leiden_scVI_1.5"].copy()

#Plot Log1p total counts
fig, ax = plt.subplots(figsize=(20,6))
sns.boxenplot(data=adata_5.obs, x="cluster", y="log1p_total_counts", ax=ax)

#Plot Pct counts mito
fig, ax = plt.subplots(figsize=(20,6))
sns.boxenplot(data=adata_5.obs, x="cluster", y="pct_counts_mito", ax=ax)

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))
sc.pl.umap(adata_5, color="GenoCre", legend_loc="right margin", ax=ax, s=10, frameon=False, save=False, palette=[])



In [ ]:
more_types=["Col14a1", "Acta2","Myh11","Tagln","Mustn1", #fibroblast
              "Lpl","Lipa","Pparg","Plin2","Ear1","Fabp1","Spp1", #lipofibroblast/osteoblastic
              "Ptprc","Mertk","Marco","Mrc1","Ly75","Adgre1","Itgax","Cd68","Csf1r","Mafb","Msr1","Arg1","Adgre4","Clec4a1", #Macs/Myeloid
              "Cx3cr1","Itgam","Cd14", #Monocytes
              "S100a9","S100a8","Mmp9","Csf3r","Cxcr2","Ly6g", #Neuts
              "Batf3","Xcr1","Clec9a","Ccl17","Ccl22", #DC
              "Cd3d","Cd3e","Cd3g","Cd28","Cd8a","Cd4","Foxp3", # Tcell
              "Gzma","Ncr1","Gzmb", #NK
              "Fcmr","Cd19","Fcer2a","Pax5","Cd22","Cd79b","Cd79a", #B cells
              "Slamf7", "Prdm1", #Plasma
              "Mcam","Pecam1","Icam2","Cd36","Cd93", #Endothelial
    "Pdpn","Cav1","Cav2","Hopx","Timp3","Sema3f","Serpine1", #AT1
              "Abca3","Muc1","Sftpa1","Sftpb","Sftpd","Scd1", #AT2
              "Scgb1a1","Cyp2f2","Scgb3a2", "Scgb3a1","Lypd2",#Club
              "Muc5ac","Muc5b", # Goblet
              "Tubb4a","Foxa3","Foxj1","Rfx2","Rfx3","Trp73", #Ciliated
              'Krt5', 'Krt17','Krt15','Trp63','Id1','Icam1','Epas1','Aqp3','Sfn','Perp','Fxyd3','Sdc1','Gstm2','F3','Abi3bp','Adh7', # Basal
              'Bex2','Ascl1','Meis2','Hes6','Hoxb5','Foxa2','Sox4','Rora','Isl1','Id4', 'Neurod1','Neurod4','Nhlh1','Nhlh2',#NE/neuronal
              'Pou2f3','Trpm5','Ascl2','Ehf',
              'Lrmp','Gng13','Ltc4s','Alox5ap','Avil','Alox5','Atp2a3','Plk2', #tuft
              "Cftr","Ascl3", 'Stap1','Atp6v1c2','Pparg','Rasd1','Slc12a2', #ionocyte
              "Gja1","Nkx2-1","Epcam", # Lung lineage
              'Yap1','Wwtr1','Sox2','Cd44','Hes1', # Stem-like
             "Venus","fLuc", "Top2a","Mki67"] # Tumor markers

sc.pl.umap(
    adata_5,
    color=more_types,
    use_raw=False,
    legend_loc= "on data",
    color_map="cmo.dense",
    ncols=4,
    vmax="p99.5",s=50,
    frameon=False,
    layer="norm",
    save=False
)

In [ ]:
more_types=["Ascl1","Neurod1","Pou2f3","Yap1"] # Tumor markers

sc.pl.umap(
    adata_5,
    color=more_types,
    use_raw=False,
    legend_loc= "on data",
    color_map="cmo.dense",
    ncols=4,
    vmax="p99.5",s=50,
    frameon=False,
    layer="norm",
    save=False
)

In [ ]:
#QC UMAPs
sc.pl.umap(
    adata_5,
    color=["n_genes_by_counts", "total_counts", "pct_counts_mito", "log1p_total_counts"],
    cmap="cubehelix_r",
    s=3,
    ncols=2,
)

In [ ]:
# Find cluster markers for each leiden cluster to aid filtering
sc.tl.rank_genes_groups(adata_5, 'leiden_scVI_1.5', method='wilcoxon', layer='norm', use_raw=False)
pd.DataFrame(adata_5.uns['rank_genes_groups']['names']).head(50)

In [ ]:
adata_5.write_h5ad("08.24.24_RPM_K5vCGRP_adata5.h5ad")

In [ ]:
adata_5=sc.read_h5ad("08.24.24_RPM_K5vCGRP_adata5.h5ad")

In [ ]:
# C8 is low quality (higher mito and n_genes_by_counts), remove and re-cluster

bad_clust=['8']

#Filter out bad clusters
to_keep=(~adata_5.obs['leiden_scVI_1.5'].isin(bad_clust))

#Copy over to new anndata object
adata_6 = adata_5[to_keep].copy()



## ITERATION 6 (Final iteration, high quality tumor cells only)

In [ ]:
# ID HVG via Scanpy

sc.pp.highly_variable_genes(
    adata_6,n_top_genes=10000,
    subset=False,
    layer="counts",
    flavor="seurat_v3",
    batch_key="Batch"
)

In [ ]:
# Create model

scvi.model.SCVI.setup_anndata(
    adata_6,
    layer="counts",
    batch_key='Batch',
    continuous_covariate_keys=["pct_counts_mito"]
)

model = scvi.model.SCVI(adata_6)
model.train()

In [ ]:
#Fit model to data
#Get latent representation of model to apply to UMAP

latent = model.get_latent_representation()

adata_6.obsm["X_scVI_1.6"] = latent


In [ ]:
# Calculate neighbors using scVI model input
sc.pp.neighbors(adata_6, use_rep="X_scVI_1.6")
sc.tl.umap(adata_6, min_dist=0.75)

# Run leiden clustering based on neighbors
sc.tl.leiden(adata_6, key_added="leiden_scVI_1.6", resolution=0.25)

In [ ]:
#QC UMAPs
sc.pl.umap(
    adata_6,
    color=["n_genes_by_counts", "total_counts", "pct_counts_mito", "log1p_total_counts"],
    cmap="cubehelix_r",
    s=3,
    ncols=2,
)

more_types=["Col14a1", "Acta2","Myh11","Tagln","Mustn1", #fibroblast
              "Lpl","Lipa","Pparg","Plin2","Ear1","Fabp1","Spp1", #lipofibroblast/osteoblastic
              "Ptprc","Mertk","Marco","Mrc1","Ly75","Adgre1","Itgax","Cd68","Csf1r","Mafb","Msr1","Arg1","Adgre4","Clec4a1", #Macs/Myeloid
              "Cx3cr1","Itgam","Cd14", #Monocytes
              "S100a9","S100a8","Mmp9","Csf3r","Cxcr2","Ly6g", #Neuts
              "Batf3","Xcr1","Clec9a","Ccl17","Ccl22", #DC
              "Cd3d","Cd3e","Cd3g","Cd28","Cd8a","Cd4","Foxp3", # Tcell
              "Gzma","Ncr1","Gzmb", #NK
              "Fcmr","Cd19","Fcer2a","Pax5","Cd22","Cd79b","Cd79a", #B cells
              "Slamf7", "Prdm1", #Plasma
              "Mcam","Pecam1","Icam2","Cd36","Cd93", #Endothelial
    "Pdpn","Cav1","Cav2","Hopx","Timp3","Sema3f","Serpine1", #AT1
              "Abca3","Muc1","Sftpa1","Sftpb","Sftpd","Scd1", #AT2
              "Scgb1a1","Cyp2f2","Scgb3a2", "Scgb3a1","Lypd2",#Club
              "Muc5ac","Muc5b", # Goblet
              "Tubb4a","Foxa3","Foxj1","Rfx2","Rfx3","Trp73", #Ciliated
              'Krt5', 'Krt17','Krt15','Trp63','Id1','Icam1','Epas1','Aqp3','Sfn','Perp','Fxyd3','Sdc1','Gstm2','F3','Abi3bp','Adh7', # Basal
              'Bex2','Ascl1','Meis2','Hes6','Hoxb5','Foxa2','Sox4','Rora','Isl1','Id4', 'Neurod1','Neurod4','Nhlh1','Nhlh2',#NE/neuronal
              'Pou2f3','Trpm5','Ascl2','Ehf',
              'Lrmp','Gng13','Ltc4s','Alox5ap','Avil','Alox5','Atp2a3','Plk2', #tuft
              "Cftr","Ascl3", 'Stap1','Atp6v1c2','Pparg','Rasd1','Slc12a2', #ionocyte
              "Gja1","Nkx2-1","Epcam", # Lung lineage
              'Yap1','Wwtr1','Sox2','Cd44','Hes1', # Stem-like
             "Venus","fLuc", "Top2a","Mki67"] # Tumor markers

sc.pl.umap(
    adata_6,
    color=more_types,
    use_raw=False,
    legend_loc= "on data",
    color_map="cmo.dense",
    ncols=4,
    vmax="p99.5",s=50,
    frameon=False,
    layer="norm",
    save=False)

In [ ]:
adata_6.obs_names_make_unique()

### Save/write to import to R and further analyze in Seurat, etc.

In [ ]:
adata_6.write_h5ad("09.04.24_RPM_K5vCGRP_adata6.h5ad")

In [ ]:
adata_6=sc.read_h5ad("09.04.24_RPM_K5vCGRP_adata6.h5ad")

In [ ]:
fig, ax = plt.subplots(figsize=(8, 7))
sc.pl.umap(adata_6, color="GenoCre", legend_loc="right margin", ax=ax, s=30, frameon=False, save=False, palette=[])

fig, ax = plt.subplots(figsize=(6, 6))
sc.pl.umap(adata_6, color="leiden_scVI_1.6", legend_loc="on data", legend_fontsize='large',ax=ax, s=10, frameon=False, save=False)


In [ ]:
more_types=["Ascl1","Neurod1","Pou2f3"] # Tumor markers

sc.pl.umap(
    adata_6,
    color=more_types,
    use_raw=False,
    legend_loc= "on data",
    color_map="cmo.dense",
    ncols=4,
    vmax="p99.5",s=60,
    frameon=False,
    layer="norm",
    save=False)

In [ ]:
#Generate signatures from these two tumors after normalizing and log transforming count data 
sc.pp.normalize_total(adata_6)
sc.pp.log1p(adata_6)
sc.tl.rank_genes_groups(adata_6, 'leiden_scVI_1.6', method='t-test')


In [ ]:
# Extract top 500 marker genes for leiden clusters from data 
sc.tl.rank_genes_groups(adata_6,'leiden_scVI_1.6', method='wilcoxon', n_genes=500)

result = adata_6.uns['rank_genes_groups']
groups = result['names'].dtype.names
markergenes=pd.DataFrame(
    {group + '_' + key[:1]: result[key][group]
    for group in groups for key in ['names', 'pvals']})


In [ ]:
markergenes.to_csv('/hpc/home/asi16/RPM_K5vCGRP_Leiden_scRNAseq_100724.csv' )

In [ ]:
#Generate signatures from these two tumors
sc.pp.normalize_total(adata_6)
sc.pp.log1p(adata_6)
sc.tl.rank_genes_groups(adata_6, 'leiden_scVI_1.6', method='t-test')


# Extract top 500 marker genes for leiden clusters from data 
sc.tl.rank_genes_groups(adata_6,'leiden_scVI_1.6', method='wilcoxon', n_genes=500)

result = adata_6.uns['rank_genes_groups']
groups = result['names'].dtype.names
markergenes=pd.DataFrame(
    {group + '_' + key[:1]: result[key][group]
    for group in groups for key in ['names', 'pvals']})
     
markergenes.to_csv('/hpc/home/asi16/RPM_K5vCGRP_Leiden_scRNAseq_100724.csv' )